In [1]:
%reload_ext autoreload
%autoreload 2


# Quick Start

To run a simulation on a terrain stored as an heightmap and store the robot's information in a bag file. Remember to throttle the `/pose` topic with
`rosrun topic_tools throttle messages /krock/pose 50`

In [2]:
import time
start = time.time()
from simulation import Simulation
from PIL import Image
import glob
image_list = []

# Run the simulation for single map

out_dir='/data/iakram/Shared_data/bumps0.png'
sim = Simulation(map_path='./maps/new-train/bumps0.png', max_time=10, n=1, out_dir=out_dir, height=1)
sim()

# Run the simulation for all type of maps

#for filename in glob.glob('./maps/new-train/*.png'): #assuming gif
#    im=Image.open(filename)
#    sim = Simulation(map_path=filename, max_time=2, n=1, out_dir=out_dir, height=1)
#    sim()

print('[INFO] bags stored at {}'.format(sim.agent_out_dir))

end = time.time()
print(end - start)

  0%|          | 0/1 [00:00<?, ?it/s]

/root/code/simulation/env/webots/krock/children_no_tail
success: 1


[INFO] bags stored at /data/iakram/Shared_data/bumps0.png/bags
14.50643539428711


You can also pass a `spawn_strategy` that decides how the robot will spawn on the ground. We provided a [`RandomSpawnStrategy`](https://github.com/FrancescoSaverioZuppichini/Master-Thesis/blob/master/core/simulation/env/spawn/SpawnStragety.py) and a [`FlatGroundSpawnStrategy`](https://github.com/FrancescoSaverioZuppichini/Master-Thesis/blob/master/core/simulation/env/spawn/SpawnStragety.py). The second will spawn the robot on the flat part of the terrain.

The previous snippet will run two simulations for 10 second each on the provided map. Krock will walk on the terrain until the time finish or it reach the boundaries of the terrain. The simulation information are stored in a `.csv` file at `out_dir/meta.csv`.

In [5]:
import pandas as pd
pd.read_csv('/data/iakram/Shared_data/meta.csv')

FileNotFoundError: [Errno 2] File b'/data/iakram/Shared_data/meta.csv' does not exist: b'/data/iakram/Shared_data/meta.csv'

The `.bag` files are stored into `tmp`

In [6]:
!ls /data/iakram/Shared_data/bags | grep .bag

ls: cannot access '/data/iakram/Shared_data/bags': No such file or directory


## Customization
To change how the robot behaviour you can create subclass the `Simulation` class and override `step`

In [ ]:
from random import randint

class SimulationWithRandomGait(Simulation):
    def step(self):
        return self.env.step({
        'frontal_freq': 5,
        'lateral_freq': 0,
        'gait': randint(1,3) # this will select a different gait at each step
    })
    
sim = SimulationWithRandomGait(map_path='./maps/new-train/bumps0.png', max_time=10,  n=2, out_dir='/tmp/', height=1)
sim()

To understand why and how this work please read the following sections.

# API

This packages provide a clean API to create and run a robotic simulation. In our work, we used Webots as engine and a robot called Krock. 

## Webots + Krock
![alt text](https://raw.githubusercontent.com/FrancescoSaverioZuppichini/Master-Thesis/master/resources/images/krock.jpg)

In order to run the Krock robot using Webots we provide a [gym](https://gym.openai.com/) env called `KrockWebotsEnv`. If you know gym, then you are good to go. It follows a **basic example** where we let Krock walk forward

In [15]:
import rospy
import pprint

from simulation.env.webots.krock import Krock, KrockWebotsEnv
# load a world file
WORLD_PATH = '/data/iakram/Master-Thesis/core/simulation/env/webots/krock/krock2_ros/worlds/bumps0.wbt'
N_STEPS = 4

rospy.init_node("traversability_simulation")
# create our env
env = KrockWebotsEnv.from_file(WORLD_PATH)
# spawn the robot at random pose
init_obs = env.reset(pose=env.random_position)

print('Initial observations:')
pprint.pprint(init_obs)

for _ in range(N_STEPS):
#   go forward for a while
    obs, r, done, _ = env.step(env.GO_FORWARD)
    pprint.pprint(obs)
    if done: break
        
env.step(env.STOP)

/root/code/simulation/env/webots/krock/children_no_tail
success: 1
Initial observations:
{'sensors': {'front_cam': None,
             'orientation': {'w': 0.12401767539506328,
                             'x': 0.03449139119744101,
                             'y': 0.0001368480154349657,
                             'z': -0.9916803625137943},
             'position': {'x': -1.7905448807601698,
                          'y': 3.98588688542524,
                          'z': -0.7806482703837692}}}
{'sensors': {'front_cam': None,
             'orientation': {'w': 0.12401767539506328,
                             'x': 0.03449139119744101,
                             'y': 0.0001368480154349657,
                             'z': -0.9916803625137943},
             'position': {'x': -1.7905448807601698,
                          'y': 3.98588688542524,
                          'z': -0.7806482703837692}}}
{'sensors': {'front_cam': None,
             'orientation': {'w': 0.12401767539506328,
    

({'sensors': {'position': {'x': -1.7905358771542075,
    'y': 3.9859074914852037,
    'z': -0.7806137180050605},
   'orientation': {'x': 0.034479980564973464,
    'y': 0.00015096423132510424,
    'z': -0.9916840241681092,
    'w': 0.12399154954989873},
   'front_cam': None}},
 0,
 False,
 {})

### Actions

`KrockWebotsEnv` implements `WebotsEnv` that is a generic env for Webots. Thanks to to gym interface we can explore the action space and the observation space easily



In [16]:
print(env.action_space)
print(env.observation_space)

Dict(frontal_freq:Box(), gait:Discrete(2), lateral_freq:Box())
Dict(sensors:Dict(front_cam:Tuple(Box(600, 800, 3), Box(600, 800, 3)), orientation:Dict(w:Box(1,), x:Box(1,), y:Box(1,), z:Box(1,)), position:Dict(x:Box(1,), y:Box(1,), z:Box(1,))))


Select a random action is even esier


In [17]:
obs, r, done, _ = env.step(env.action_space.sample())
env.step(env.STOP)

({'sensors': {'position': {'x': -1.864495567480065,
    'y': 3.969835805996932,
    'z': -0.7994761529604703},
   'orientation': {'x': 0.016735405762523494,
    'y': 0.0002986041413179069,
    'z': -0.9883825942399058,
    'w': 0.15106185631430571},
   'front_cam': None}},
 0,
 False,
 {})

We can easily **change the gait** by passing the correct action
![alt text](https://raw.githubusercontent.com/FrancescoSaverioZuppichini/Master-Thesis/master/resources/images/krock-gait.jpg)


In [19]:
import time
# init_obs = env.reset(pose=env.random_position)
obs, r, done, _ = env.step({
        'frontal_freq': 5,
        'lateral_freq': 0,
        'gait': 2 # change gait to 2
    })
time.sleep(2)
obs, r, done, _ = env.step(env.STOP) # stop will also reset the gait to 1

# Under the hood

To create the final enviroment we had to first.

## Define the Enviroment
Under the hood `KrockWebotsEnv` implements [`WebotsEnv`](https://github.com/FrancescoSaverioZuppichini/Master-Thesis/blob/master/core/simulation/env/webots/WebotsEnv.py). `WebotsEnv` creates the enviroment by from an image (heightmap)/ numpy array or a webots worl file. Check the `classmethod`. It also allows to get the height in a point and to spawn the robot in the terrain. 

## Define the Robot

### Agent API
The base class to create a robot is `Agent`

In [20]:
from simulation.agent import Agent

class DummyAgent(Agent):
    def __call__(self, *args, **kwargs):
        print('I am alive!')
    def act(self, env, *args, **kwargs):
        print('I am moving!')
    def die(self, *args, **kwargs):
        print('bye bye')
    
a = DummyAgent()
env = None
a() # init
a.act(env) # do something
a.die(env) # RIP, clean up (e.g store stuff empty the state)

I am alive!
I am moving!
bye bye


### ROS
A `Rospy` agent is already implemented and can be found at `agent.RospyAgent`. It adds two methods to the basic `Agent`: `init_publishers` and `init_subscribers`. They must be overrided to initialise the subscribers and listeners. 

In [48]:
from simulation.agent import RospyAgent
from geometry_msgs.msg import PoseStamped
from sensor_msgs.msg import Joy

class MyRospyAgent(RospyAgent):
    def init_publishers(self):
        return {
            'joy': rospy.Publisher('/joy', Joy, queue_size=1),
        }

    def init_subscribers(self):
        return { 'pose': 
                rospy.Subscriber('/pose', PoseStamped, self.callback_pose)
               }

    def callback_pose(self, data):
        # this will triggers the `on_state_change` method in all callbacks
        self.state['pose'] = data

a = MyRospyAgent()
a()
print(a.subscribers)
print(a.publishers)


{'pose': <rospy.topics.Subscriber object at 0x7fe39f1a9978>}
{'joy': <rospy.topics.Publisher object at 0x7fe39f1a9b38>}


### Callbacks
`Agent` implements the protocol `Callbackable` protocol. A callback can be
added by calling the `add_callback` or `add_callbacks`. 

In [49]:
from simulation.agent.callbacks import AgentCallback

class MyCallback(AgentCallback):
    def on_state_change(self, agent, key, value):
        print(key, value)

Each `Agent` has a `state` that is just a `dict` . Every time it is changed it triggers the `on_state_change` event. This is usefull
to record stuff.

In [50]:
a = DummyAgent()
a.add_callback(MyCallback())
a()

a.state['foo'] = 'foo'

I am alive!
foo foo


#### RosBagSaver

A callback to store the topics of an agent into bags file is implemented

In [51]:
from simulation.agent.callbacks import RosBagSaver

a = MyRospyAgent()
a.add_callbacks([RosBagSaver(save_dir='/temp/data', topics=['pose'])])
a()

The files will be stored everytime `agent.die()` il called. A new thread is always spawned to avoid blocking

## Krock
[`Krock`](https://github.com/FrancescoSaverioZuppichini/Master-Thesis/blob/master/core/simulation/env/webots/krock/Krock.py) is just a `RospyAgent` that has already all the subscribers and listeners.